In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import sdeint

# Define constants
m = 1.0
omega = 9.0
gamma = 1.0
c = 1.0
EFeld = 7.0

#Use if inputting different constants is necessary
#m = input("Enter value for m")
#omega = input("Enter value for omega")
#gamma = input("Enter value for gamma")
#c = input("Enter value for c")
#EFeld = input("Enter value for EFeld")
 
 

# Define the system of ODEs (without noise)
def deterministic_system(Y, t):
    qx, qy, qz, px, py, pz = Y
    dqx_dt = px / m
    dqy_dt = py / m
    dqz_dt = pz / m
    dpx_dt = -gamma * px - omega * py
    dpy_dt = -gamma * py + omega * px + EFeld
    dpz_dt = -gamma * pz
    return np.array([dqx_dt, dqy_dt, dqz_dt, dpx_dt, dpy_dt, dpz_dt])  # Return in the form of a NumPy array so sdient can work

# Define noise terms (Wiener processes) with correct shape
def noise_system(Y, t):
    # Create a zero matrix with shape (6, 6)
    noise_matrix = np.zeros((6, 6))
    
    # Add noise to the momentum components (px, py, pz)
    noise_matrix[3, 3] = c
    noise_matrix[4, 4] = c
    noise_matrix[5, 5] = c
    
    return noise_matrix

# Initial conditions
Y0 = np.array([0.0, 0.0, 0.0, 0.2, 0.2, 0.2])  # Also as a NumPy array

# Time array
t = np.linspace(0, 20, 20001)

# Simulate the system
result = sdeint.itoint(deterministic_system, noise_system, Y0, t)

# Extract positions
qx = result[:, 0]
qy = result[:, 1]
qz = result[:, 2]

# Plot 3D trajectory
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot(qx, qy, qz)
ax.set_title("3D Trajectory")
ax.set_xlabel('qx')
ax.set_ylabel('qy')
ax.set_zlabel('qz')
plt.show()

# Plot individual components over time
plt.figure()
plt.plot(t, qx, label='qx', color='green')
plt.plot(t, qy, label='qy', color='red')
plt.plot(t, qz, label='qz', color='blue')
plt.title("Position Components Over Time")
plt.xlabel('Time')
plt.ylabel('Position')
plt.legend()
plt.show()

# Autocorrelation function
def autocorrelation(x):
    result = np.correlate(x, x, mode='full')
    return result[result.size // 2:]

acf_qx = autocorrelation(qx)
acf_qy = autocorrelation(qy)
acf_qz = autocorrelation(qz)

# Plot autocorrelation functions
plt.figure()
plt.plot(acf_qx, label='ACF qx', color='green')
plt.plot(acf_qy, label='ACF qy', color='magenta')
plt.plot(acf_qz, label='ACF qz', color='blue')
plt.title("Autocorrelation Functions")
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.legend()
plt.show()